In [1]:
#fast.ai and PyTorch import
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from fastai.text import *
from fastai.tabular import *

import glob


#dataframes and such
import numpy as np
import pandas as pd

#Graphing
import matplotlib.pyplot as plt
import matplotlib as plt
import plotly   #plotly for graphs
import plotly.express as px
import plotly.graph_objects as go

from datetime import datetime
import datetime as dt  #datetime for date formatting of indices

import sklearn
from sklearn import linear_model 
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing


import random
import csv
import itertools
import pathlib



In [727]:
#CREATE A DATASET CLASS - feed into databunch.create to make dataloaders
'''
this is me going into the pytorch source code to remake a dataset type
the goal being to output a sample in the 3D format
'''
#############################################


import bisect
import warnings
from torch._utils import _accumulate
from torch import randperm


class MyDataset(Dataset):
    """create dataset class specifically for nasdaq100
    potentially create staggered column in this method?
    read from csv?
    
    introduce normalize function?
    """

    def __init__(self, filename, path, target, tp, bptt, split):  #read the df in the start  (bs? bptt?)
        """
        Args:
            filename: csv file containing data
            path: from root directory to location of file
            target:  target stock code (NDX)
            tp:  number of time periods into the future that you want to predict
            bptt:  number of objects per item
            
        """
        
        df_full = pd.read_csv(path/filename) #pull data from filename

        # create date range from 26/07/16 - 22/12/16 in 40,560 rows
        date_rng = pd.date_range(start='26/07/2016', end='23/12/2016', periods=40560); # 5:19.5(mins)
        df_full.insert(0, 'datetime', date_rng)  #create date column
        df_full = df_full.set_index('datetime')  #set datetune to index

        #CREATE STAGGERED TARGET COLUMN
        target_list = df_full[target].tolist()  #creates list from target column
        target_list.extend([np.nan for x in range(tp)])  #extends list with NaNs for specified time period
        target_list = target_list[tp:]  #cuts the list back to size
        
        #Normalize data
        x = df_full  #.values #returns a numpy array
        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(x)
        df_full = pd.DataFrame(x_scaled)
        
        #unnormalized target column inserted after rest of df is normalized
        df_full.insert(0, 'NDX (Target)', target_list) #insert target data
        df_full = df_full.dropna()  #dataset will be shortened by tp
        
        
        #REDUCE df_nasdaq100 INTO THE SMALLER, SHITTIER df
        df = df_full
        '''.iloc[:, 0:5]  #df_shit is only first 5 columns of full df
        df.insert(1, 'NDX',  df_nasdaq100['NDX'].values.tolist())  #insert ndx column'''

        if split   == "full" :  self.data = df   #mode 0: full df
        elif split == "train":  self.data = df[:-5460]    #mode 1: train set
        elif split == "valid":  self.data = df[-5460:-2730]  #mode 2: valid set
        elif split == "test" :  self.data = df[-2730:] #mode 3: test set

            
            

    def __len__(self):
        return len(self.data)


    #one for the bptt
    '''def __getitem__(self, idx):  #RETURNS A SINGLE ITEM, IRRESPECTIVE OF BATCH
        #ONE ITEM CONSISTS OF <bptt> OBJECTS
        X_tt, Y_tt = None, None  #initialises tensors
        
        Y_temp = self.data.iloc[idx].values.tolist()[0]
        Y_tt = torch.unsqueeze(torch.tensor(Y_temp), dim=0)  #FORMAT CAN BE CHANGED IF NEED BE
        
        if idx+1 < bptt:  #if requested item length less than available rows, return shotened
            for i in range(idx+1):  #0 to idx-1  for a sequence of idx values
                row = self.data.iloc[i].values.tolist()
                X_temp = torch.unsqueeze(torch.tensor(row[1:]), dim=0)  #input
                try:
                    X_tt = torch.cat((X_tt, X_temp), dim=0)  #create sample tensor
                except:
                    X_tt = X_temp
        
        elif idx+1 >= bptt:     #can't take sequence -2,-1,0, so only starts late
            for i in range(bptt):
                row = self.data.iloc[idx-i].values.tolist()
                X_temp = torch.unsqueeze(torch.tensor(row[1:]), dim=0)  #input
                try:
                    X_tt = torch.cat((X_temp, X_tt), dim=0)  #create sample tensor
                except:
                    X_tt = X_temp
        return X_tt, Y_tt'''
    
    
    #one for a simple output
    '''def __getitem__(self, idx):  #RETURNS A SINGLE ITEM, IRRESPECTIVE OF BATCH
        #ONE ITEM CONSISTS OF a single OBJECT
        row = self.data.iloc[idx].values.tolist()
        Y_tt = torch.tensor(row[0])  #FORMAT CAN BE CHANGED IF NEED BE
        X_tt = torch.tensor(row[1:])  #input
        return X_tt.unsqueeze(0), Y_tt.unsqueeze(0)  #X in [sequence, batch, feature] format'''
    
    
    #return a single object
    def __getitem__(self, idx):  #RETURNS A SINGLE object
        row = self.data.iloc[idx].values.tolist()
        Y_tt = torch.tensor(row[0])  #FORMAT CAN BE CHANGED IF NEED BE
        X_tt = torch.tensor(row[1:])  #input
        return X_tt, Y_tt  #X in [sequence, batch, feature] format

    


In [731]:
#define loss function
def lossFlat(p,t):
    return loss_fn(p.flatten(), t.flatten())
loss_fn = nn.MSELoss()  #mse loss function


criterion = torch.nn.SmoothL1Loss()
def loss4(input, target):
    return criterion(input, target)


#define the lstm model
class LSTMSimpleMdl(nn.Module):
    def __init__(self, ni, nh, nl, aInput):
        # ni - number of input features
        # nh - number of hidden features
        # nl - number of stacked LSTM layers
        # aInput - True: append input before sending to linear (an experiment)

        super().__init__()
        self.nl = nl
        self.nh = nh
        self.ni = ni
        self.aInput = aInput

        self.lstm1 = nn.LSTM(self.ni, self.nh, self.nl, batch_first=True)
        self.linear = nn.Linear(self.nh, 1)
        
        #current sequence length is 1,  can this can be bumped up as bptt??

    def forward(self, input):
        ninput = input
        h_t = torch.zeros(self.nl, input.shape[0], self.nh, dtype=torch.float)#.cuda()  # hidden state for each batch element
        c_t = torch.zeros(self.nl, input.shape[0], self.nh, dtype=torch.float)#.cuda()
        output, (h_t, c_t) = self.lstm1(ninput, (h_t, c_t))
        
        output = self.linear(output)
        return output.flatten(1)


model = LSTMSimpleMdl(82, 100, 1, False)#.cuda()

In [732]:



####


####




In [733]:
#ESTABLISH PROJECT CONDITIONS
filename = 'nasdaq100.csv'     # data file to be used
path = pathlib.Path.home()/'OneDrive'/'19-20 3rd Yr MEng'/'Dissertation'/'Data'/'NASDAQ-100 II'
target = 'NDX'  #target column name to be predicted
tp = 10  #time periods into the future we're tring to predict
bptt = 20  #number of objects per sequence
bs = 1  # number of batches that the dataset is split into


#no normalization is happening rn, smh
train_ds = MyDataset(filename, path, target, tp, bptt, "train")  #create train dataset
valid_ds = MyDataset(filename, path, target, tp, bptt, "valid")  #create valid dataset
test_ds  = MyDataset(filename, path, target, tp, bptt, "test")  #create valid dataset

train_dl = DataLoader(train_ds, batch_size=bs, shuffle=False, num_workers=0)
valid_dl = DataLoader(valid_ds, batch_size=bs, shuffle=False, num_workers=0)
test_dl  = DataLoader(test_ds , batch_size=bs, shuffle=False, num_workers=0)

data = DataBunch(train_dl, valid_dl, None, test_dl)

In [734]:
x,y = data.one_batch()
print(x.size())
#model.forward(x)


torch.Size([1, 82])


In [703]:
it = iter(data.valid_dl)
x1,y1 = next(it)
x2,y2 = next(it)
x3,y3 = next(it)
it.close()

In [735]:
x1.size()

torch.Size([1, 82])

In [640]:



###


###




DeviceDataLoader(dl=<torch.utils.data.dataloader.DataLoader object at 0x0000020B4CD2D5F8>, device=device(type='cpu'), tfms=[], collate_fn=<function data_collate at 0x0000020B38B0AAE8>)

In [489]:
#define training loop
def trainN(N,lr):
    global test_pred,vmtarget,output,mtarget,mbatch,vmbatch
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for i in range(N):   
        for mbatch,mtarget in train_dl:
            optimizer.zero_grad()
            output = model.forward(mbatch)
            mtarget = mtarget[:,-output.shape[1]:] #shorten target to match shortened output
            loss = lossFlat(output, mtarget)
            if i%1==0:
                loss.backward()
                optimizer.step()

        with torch.no_grad():
            for vmbatch,vmtarget in valid_dl:
                test_pred = model(vmbatch)
                vmtarget = vmtarget[:,-test_pred.shape[1]:] #shorten target to match shortened output
                vloss = lossFlat(test_pred, vmtarget)
        print('%i %2.9f %2.9f' % (i, loss.item(), vloss.item()))

trainN(30,.01)

0 5355280.000000000 6362075.500000000
1 246315.656250000 496648.968750000
2 4367.250488281 20430.333984375
3 20.321662903 40870.824218750
4 546.791992188 53901.375000000
5 265.955657959 35191.984375000
6 579.258422852 33585.750000000
7 26.987842560 45704.058593750
8 97.020751953 47760.031250000
9 778.921630859 56023.859375000
10 134.693542480 48552.906250000
11 189.878509521 49499.281250000
12 111.608230591 48074.472656250
13 159.578308105 48983.554687500
14 8.426210403 44804.789062500
15 448.517028809 52808.898437500
16 2251.742919922 26298.005859375
17 3497.498535156 22591.146484375
18 0.240095824 43547.292968750
19 3758.528808594 21965.912109375
20 4525.682128906 20197.919921875
21 4114.554687500 21124.908203125
22 4368.805664062 20552.078125000
23 4044.482177734 21015.218750000
24 2.592918396 38065.960937500
25 3.806054354 42829.109375000
26 4890.383300781 19440.117187500
27 4509.593750000 20245.824218750
28 4608.960937500 20032.841796875
29 4818.534179688 19592.212890625


In [564]:
data = DataBunch(train_dl, valid_dl, None, test_dl)  #needs those data loaders!
learn = Learner(data, model, loss_func=loss4)  #lossFlat or loss4
learn.fit_one_cycle(10, max_lr=10, wd=0)

epoch,train_loss,valid_loss,time
0,25.791744,101.262726,00:16
1,29.229778,70.027428,00:18
2,51.148079,98.329262,00:21
3,88.007874,249.098190,00:21
4,38.525616,152.367233,00:21
5,25.847212,112.136757,00:21
6,17.577890,79.129387,00:22
7,15.843431,76.796516,00:21
8,9.220344,72.848625,00:21
9,43.206398,69.615089,00:21


In [565]:
##create a method for displaying predictions

test_df = test_ds.data
preds = learn.get_preds(ds_type = DatasetType.Test)
list_pred = preds[0].squeeze().tolist()
test_df.insert(1, 'NDX Prediction', list_pred)

In [566]:
test_df

,NDX (Target),NDX Prediction,0,1,2,3,4,5,6,7,...,72,73,74,75,76,77,78,79,80,81
37820,4949.261,4764.152344,0.849954,0.865073,0.720813,0.905630,0.744656,0.885445,0.760308,0.017044,...,0.768623,0.123316,0.186653,0.173501,0.937346,0.857143,0.532117,0.660484,0.518142,0.980307
37821,4949.513,4764.152344,0.850147,0.865186,0.718889,0.905310,0.745234,0.886595,0.761262,0.023261,...,0.774266,0.125766,0.186667,0.173346,0.939957,0.856795,0.537226,0.661290,0.518142,0.980560
37822,4950.048,4764.152344,0.850147,0.866245,0.721454,0.904671,0.745812,0.885062,0.761175,0.021794,...,0.775395,0.125504,0.185333,0.173035,0.945178,0.857122,0.540876,0.661290,0.518142,0.980129
37823,4950.594,4764.152344,0.849577,0.868723,0.723378,0.904990,0.741768,0.885445,0.761175,0.021514,...,0.774266,0.127399,0.185773,0.173346,0.946048,0.858533,0.540146,0.660484,0.519594,0.980786
37824,4949.426,4764.152344,0.849577,0.866695,0.720813,0.905310,0.741190,0.885062,0.761769,0.021512,...,0.774266,0.126582,0.186000,0.171482,0.947788,0.858881,0.540175,0.658871,0.513788,0.976844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40545,4921.811,4764.152344,0.889464,0.872710,0.564017,0.900192,0.979203,0.690435,0.853071,0.329142,...,0.846501,0.061249,0.072667,0.020503,0.680639,0.954119,0.633387,0.920161,0.071118,0.871527
40546,4921.580,4764.152344,0.889179,0.873859,0.564979,0.899552,0.975737,0.688136,0.852637,0.329422,...,0.846501,0.060024,0.073333,0.018639,0.678899,0.952729,0.630657,0.918548,0.069666,0.869545
40547,4921.884,4764.152344,0.886615,0.871201,0.557925,0.896353,0.975159,0.688902,0.850470,0.331657,...,0.842551,0.061249,0.073880,0.018329,0.679769,0.954119,0.629927,0.916935,0.065312,0.865411
40548,4922.949,4764.152344,0.885475,0.874080,0.558970,0.896353,0.974003,0.689285,0.851510,0.329701,...,0.843115,0.061658,0.074120,0.018950,0.675418,0.954466,0.631387,0.915323,0.067489,0.866678


In [567]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=test_df.index, y=test_df['NDX (Target)'],
    fill=None,
    mode='lines',
    name='Target (^NDX Close)'
    ))

fig.add_trace(go.Scatter(x=test_df.index, y=test_df['NDX Prediction'],
    fill=None,   #'tonexty',  #fill area between trace0 and trace1
    mode='lines',
    name='Prediction (^NDX Close)'
    ))

fig.show()

In [13]:
#Test dataset class
'''
this is me going into the pytorch source code to remake a dataset type
the goal being to output a sample in the 3D format
'''
#############################################


import bisect
import warnings
from torch._utils import _accumulate
from torch import randperm


class Trial_Dataset(Dataset):
    """create dataset class specifically for nasdaq100
    potentially create staggered column in this method?
    read from csv?
    
    introduce normalize function?
    """

    def __init__(self, filename, path, target, tp, bptt, split):  #read the df in the start  (bs? bptt?)
        """
        Args:
            filename: csv file containing data
            path: from root directory to location of file
            target:  target stock code (NDX)
            tp:  number of time periods into the future that you want to predict
            bptt:  number of objects per item
            
        """
        
        df_full = pd.read_csv(path/filename) #pull data from filename

        # create date range from 26/07/16 - 22/12/16 in 40,560 rows
        date_rng = pd.date_range(start='26/07/2016', end='23/12/2016', periods=40560); # 5:19.5(mins)
        df_full.insert(0, 'datetime', date_rng)  #create date column
        df_full = df_full.set_index('datetime')  #set datetune to index

        #CREATE STAGGERED TARGET COLUMN
        target_list = df_full[target].tolist()  #creates list from target column
        target_list.extend([np.nan for x in range(tp)])  #extends list with NaNs for specified time period
        target_list = target_list[tp:]  #cuts the list back to size
        
        #Normalize data
        x = df_full  #.values #returns a numpy array
        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(x)
        df_full = pd.DataFrame(x_scaled)
        
        #unnormalized target column inserted after rest of df is normalized
        df_full.insert(0, 'NDX (Target)', target_list) #insert target data
        df_full = df_full.dropna()  #dataset will be shortened by tp
        
        
        #REDUCE df_nasdaq100 INTO THE SMALLER, SHITTIER df
        df = df_full
        '''.iloc[:, 0:5]  #df_shit is only first 5 columns of full df
        df.insert(1, 'NDX',  df_nasdaq100['NDX'].values.tolist())  #insert ndx column'''

        if split   == "full" :  self.data = df[:]   #mode 0: full df
        elif split == "train":  self.data = df[:-5460]    #mode 1: train set
        elif split == "valid":  self.data = df[-5460:-2730]  #mode 2: valid set
        elif split == "test" :  self.data = df[-2730:] #mode 3: test set
            
            

    def __len__(self):
        return len(self.data)


    
    #return a single object
    def __getitem__(self, idx):  #RETURNS A SINGLE object
        row = self.data.iloc[idx].values.tolist()
        Y_tt = torch.tensor(row[0])  #FORMAT CAN BE CHANGED IF NEED BE
        X_tt = torch.tensor(row[1:])  #input
        return X_tt.unsqueeze(0), Y_tt.unsqueeze(0)  #a tuple,  x in the form [1,82]


    
    
    
#######################################


        
        
#######################################

def collate_fn(data):
    """
       data: is a list of tuples with (example, label, length)
             where 'example' is a tensor of arbitrary shape
             and label/length are scalars
    """
    inputs, labels = zip(*data)
    max_len = max(lengths)
    n_ftrs = data[0][0].size(1)
    features = torch.zeros((len(data), max_len, n_ftrs))
    labels = torch.tensor(labels)
    lengths = torch.tensor(lengths)

    for i in range(len(data)):
        j, k = data[i][0].size(0), data[i][0].size(1)
        features[i] = torch.cat([data[i][0], torch.zeros((max_len - j, k))])

    return features.float(), labels.long(), lengths.long()        
        
##################################

filename = 'nasdaq100.csv'     # data file to be used
path = pathlib.Path.cwd()
target = 'NDX'  #target column name to be predicted
tp = 10  #time periods into the future we're tring to predict
bptt = 20  #number of objects per sequence

bs = 8  # number of batches that the dataset is split into
seq_len = None

t_ds = Trial_Dataset(filename, path, target, tp, bptt, "train")  #create train dataset
v_ds = Trial_Dataset(filename, path, target, tp, bptt, "valid")  #create valid dataset
test_ds = Trial_Dataset(filename, path, target, tp, bptt, "test")  #create valid dataset

t_dl  = DataLoader(t_ds , batch_size=len(t_ds), shuffle=False, num_workers=0)
v_dl  = DataLoader(v_ds , batch_size=len(v_ds), shuffle=False, num_workers=0)
test_dl  = DataLoader(test_ds , batch_size=len(test_ds), shuffle=False, num_workers=0)

data = DataBunch(t_dl, v_dl, None, test_dl)

x,y = data.one_batch()
print(x.size(), y.size())
print(x)


torch.Size([35090, 1, 82]) torch.Size([35090, 1])
tensor([[[0.1675, 0.0164, 0.1501,  ..., 0.0065, 0.0000, 0.0508]],

        [[0.1647, 0.0205, 0.1501,  ..., 0.0169, 0.0189, 0.0508]],

        [[0.1632, 0.0232, 0.1353,  ..., 0.0371, 0.0203, 0.0533]],

        ...,

        [[0.7630, 0.5994, 0.2572,  ..., 0.4322, 0.3222, 0.3029]],

        [[0.7641, 0.5997, 0.2559,  ..., 0.4323, 0.3208, 0.3032]],

        [[0.7675, 0.6019, 0.2610,  ..., 0.4343, 0.3222, 0.3081]]])


In [14]:
#define the lstm model
class trial_model(nn.Module):
    def __init__(self, ni, nh, nl, aInput):
        # ni - number of input features
        # nh - number of hidden features
        # nl - number of stacked LSTM layers
        # aInput - True: append input before sending to linear (an experiment)

        super().__init__()
        self.nl = nl
        self.nh = nh
        self.ni = ni
        self.aInput = aInput

        self.lstm1 = nn.LSTM(self.ni, self.nh, self.nl, batch_first=False)
        self.linear = nn.Linear(self.nh, 1)
        
        #current sequence length is 1,  can this can be bumped up as bptt??

    def forward(self, input):
        ninput = input
        h_t = torch.zeros(self.nl, input.shape[1], self.nh, dtype=torch.float).cuda()  # hidden state for each batch element
        c_t = torch.zeros(self.nl, input.shape[1], self.nh, dtype=torch.float).cuda()
        output, (h_t, c_t) = self.lstm1(ninput, (h_t, c_t))
        
        output = self.linear(output)
        return output.flatten(1)


model = trial_model(82, 100, 1, False).cuda()

In [15]:
criterion = torch.nn.SmoothL1Loss()
def loss4(input, target):
    return criterion(input, target)



learn = Learner(data, model, loss_func=loss4)  #lossFlat or loss4
learn.fit_one_cycle(20, max_lr=100)

epoch,train_loss,valid_loss,time
0,4802.350098,4599.148926,00:07
1,4686.262207,4153.165039,00:07
2,4495.737305,3108.402100,00:08
3,4131.311035,1833.185669,00:07
4,3647.239258,975.540466,00:07
5,3174.363770,592.903381,00:07
6,2778.697510,295.793304,00:07
7,2442.268311,50.754898,00:07
8,2154.056152,1017.708984,00:08
9,2026.818726,636.313599,00:07


In [16]:
learn.lr_find()
learn.recorder.plot()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


KeyboardInterrupt: 

In [17]:
test_df = test_ds.data
preds = learn.get_preds(ds_type = DatasetType.Test)
list_pred = preds[0].squeeze().tolist()
test_df.insert(1, 'NDX Prediction', list_pred)



In [18]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=test_df.index, y=test_df['NDX (Target)'],
    fill=None,
    mode='lines',
    name='Target (^NDX Close)'
    ))

fig.add_trace(go.Scatter(x=test_df.index, y=test_df['NDX Prediction'],
    fill=None,   #'tonexty',  #fill area between trace0 and trace1
    mode='lines',
    name='Prediction (^NDX Close)'
    ))

fig.show()

In [839]:
test_df

,NDX (Target),NDX Prediction,0,1,2,3,4,5,6,7,...,72,73,74,75,76,77,78,79,80,81
37820,4949.261,0.497580,0.849954,0.865073,0.720813,0.905630,0.744656,0.885445,0.760308,0.017044,...,0.768623,0.123316,0.186653,0.173501,0.937346,0.857143,0.532117,0.660484,0.518142,0.980307
37821,4949.513,0.839736,0.850147,0.865186,0.718889,0.905310,0.745234,0.886595,0.761262,0.023261,...,0.774266,0.125766,0.186667,0.173346,0.939957,0.856795,0.537226,0.661290,0.518142,0.980560
37822,4950.048,1.039763,0.850147,0.866245,0.721454,0.904671,0.745812,0.885062,0.761175,0.021794,...,0.775395,0.125504,0.185333,0.173035,0.945178,0.857122,0.540876,0.661290,0.518142,0.980129
37823,4950.594,1.156542,0.849577,0.868723,0.723378,0.904990,0.741768,0.885445,0.761175,0.021514,...,0.774266,0.127399,0.185773,0.173346,0.946048,0.858533,0.540146,0.660484,0.519594,0.980786
37824,4949.426,1.225601,0.849577,0.866695,0.720813,0.905310,0.741190,0.885062,0.761769,0.021512,...,0.774266,0.126582,0.186000,0.171482,0.947788,0.858881,0.540175,0.658871,0.513788,0.976844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40545,4921.811,1.292938,0.889464,0.872710,0.564017,0.900192,0.979203,0.690435,0.853071,0.329142,...,0.846501,0.061249,0.072667,0.020503,0.680639,0.954119,0.633387,0.920161,0.071118,0.871527
40546,4921.580,1.292927,0.889179,0.873859,0.564979,0.899552,0.975737,0.688136,0.852637,0.329422,...,0.846501,0.060024,0.073333,0.018639,0.678899,0.952729,0.630657,0.918548,0.069666,0.869545
40547,4921.884,1.292180,0.886615,0.871201,0.557925,0.896353,0.975159,0.688902,0.850470,0.331657,...,0.842551,0.061249,0.073880,0.018329,0.679769,0.954119,0.629927,0.916935,0.065312,0.865411
40548,4922.949,1.292082,0.885475,0.874080,0.558970,0.896353,0.974003,0.689285,0.851510,0.329701,...,0.843115,0.061658,0.074120,0.018950,0.675418,0.954466,0.631387,0.915323,0.067489,0.866678
